In [13]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import numpy as np
import matplotlib.pyplot as plt
import spacy
import seaborn as sns
import nltk
from wordcloud import WordCloud
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [14]:
!python -m spacy download el_core_news_sm

2023-07-16 10:29:29.802928: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 120.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('el_core_news_sm')


In [15]:
nlp = spacy.load('el_core_news_sm')

In [16]:
cv = CountVectorizer(stop_words= list(nlp.Defaults.stop_words), min_df=0.01, max_df=0.95)

In [17]:
start_page = 0
end_page = 2

start_url = 'https://www.skai.gr/search/node?keys=%CE%B1%CE%BD%CF%84%CE%B9%CE%B5%CE%BC%CE%B2%CE%BF%CE%BB%CE%B9%CE%B1%CF%83%CF%84%CE%AD%CF%82&page='

test_url = start_url + str(start_page)

test_url

'https://www.skai.gr/search/node?keys=%CE%B1%CE%BD%CF%84%CE%B9%CE%B5%CE%BC%CE%B2%CE%BF%CE%BB%CE%B9%CE%B1%CF%83%CF%84%CE%AD%CF%82&page=0'

In [18]:
response = requests.get(test_url)
doc = BeautifulSoup(response.text, 'html.parser')

In [19]:
teaser_articles_list = doc.find('div', {'class':'list row padded border-box'}).find_all('article')

In [20]:
teaser_articles_list

[<article>
 <a class="img-container left" href="https://www.skai.gr/news/greece/volos-ereyna-gia-ton-giatro-pou-edine-kokteil-vitaminon-gia-ton-covid">
 <img alt="γιατρός" height="220" loading="lazy" src="/sites/default/files/styles/article_340x220/public/articles/2015m/giatros-geniki-20181124_0.jpg?itok=CPGp2yUo" title="Βόλος: Έρευνα για τον αντιεμβολιαστή γιατρό που έδινε «κοκτέιλ» βιταμινών για τον Covid" typeof="foaf:Image" width="340"/>
 </a>
 <div class="article-data border-box right">
 <h2><a href="https://www.skai.gr/news/greece/volos-ereyna-gia-ton-giatro-pou-edine-kokteil-vitaminon-gia-ton-covid">Βόλος: Έρευνα για τον αντιεμβολιαστή γιατρό που έδινε «κοκτέιλ» βιταμινών για τον Covid</a></h2>
 <time datetime="2022-12-08T08:09:51+02:00">08:09, 08.12.2022</time><p>Έρευνες σε δικαστικό επίπεδο γίνονται και στον Βόλο, για τη δράση του 73χρονου Βολιώτη αντιεμβολιαστή γιατρού, που έδινε σε ασθενείς με Covid «κοκτέιλ» βιταμινών, ...</p>
 <a class="more uppercase blue bold" href="http

In [21]:
link = teaser_articles_list[0].find('a', {'class': 'img-container left'})['href']

link

'https://www.skai.gr/news/greece/volos-ereyna-gia-ton-giatro-pou-edine-kokteil-vitaminon-gia-ton-covid'

In [22]:
teasers_list = []

for page_num in range(start_page, end_page+1):
  page_url = start_url + str(page_num)
  response = requests.get(page_url)
  doc = BeautifulSoup(response.text, 'html.parser')

  #*** ΔΩΣΤΕ ΤΗ ΔΙΚΗ ΣΑΣ ΤΙΜΗ στo teaser_articles_list***
  teaser_articles_list = doc.find('div', {'class':'list row padded border-box'}).find_all('article')


  for teaser in teaser_articles_list:
      story_dict = {}
      try:

        #*** ΔΩΣΤΕ ΤΗ ΔΙΚΗ ΣΑΣ ΤΙΜΗ στο link, ΜΕΤΑ το teaser
        link = teaser.find('a', {'class': 'img-container left'})['href']


        if link.startswith("http"):
          story_dict['url'] = link
        else:
          story_dict['url'] = main_url + link
      except:
        story_dict['url'] = ''
        pass
      teasers_list.append(story_dict)
  time.sleep(1)

In [23]:
skai_teasers_df = pd.DataFrame(teasers_list)
skai_teasers_df

,url
0,https://www.skai.gr/news/greece/volos-ereyna-g...
1,https://www.skai.gr/news/world/nea-zilandia-go...
2,https://www.skai.gr/news/greece/amvrosios-kata...
3,https://www.skai.gr/news/health/dw-treis-aione...
4,https://www.skai.gr/news/world/aystria-thrinos...
5,https://www.skai.gr/news/greece/se-syllipseis-...
6,https://www.skai.gr/news/greece/prosagoges-ant...
7,https://www.skai.gr/news/health/koronoios-ipa-...
8,https://www.skai.gr/news/politics/pleyris-diad...
9,https://www.skai.gr/news/greece/koronoios-pley...


In [24]:
skai_teasers_df.to_csv('/content/temp/skai_teasers.csv', index=False)

In [25]:
article_url = 'https://www.skai.gr/news/opinions/i-via-kai-i-tromokratia-einai-akomi-edo'

In [26]:
response = requests.get(article_url)
doc = BeautifulSoup(response.text, 'html.parser')

In [27]:
article = doc.find('body', {'class': 'skinEnabled'})

In [28]:
section = doc.find('div', {'class': 'bold uppercase'}).text
section

' Αρχική / Απόψεις '

In [29]:
title = article.find('article').find('h1').text
title

'Η βία και η τρομοκρατία είναι ακόμη εδώ...'

In [30]:
date = article.find('time')['datetime']
date

'2022-06-28T07:28:20+03:00'

In [31]:
author = article.find('a', {'class': 'author bold'}).text
author

'Του Άρη Πορτοσάλτε'

In [32]:
paragraphs = article.find_all('p')
paragraphs

[<p class="row border-box bold brd-bt pb-20 mb-20 mt-20">Η διαδήλωση αντιεμβολιαστών έξω από την κατοικία του υπουργού Υγείας Θ. Πλεύρη, το απόγευμα της Δευτέρας 27 Ιουνίου, υποδηλώνει την ακόμη μεγαλύτερη κατρακύλα της πολιτικής αντιπαράθεσης, με τα κόμματα της αντιπολίτευσης να μην αντιδρούν ακαριαία στην καταδίκη της ενέργειας.
 </p>,
 <p>Θα ζήσουμε προεκλογική περίοδο με διαμαρτυρόμενους έξω από τα σπίτια των υπουργών και την σύζυγο με τα παιδιά τους εντός κατοικίας;</p>,
 <p>Είναι μέρος της πολιτικής αντιπαράθεσης η <strong>στοχοποίηση της οικογένειας του υπουργού</strong>; Επικροτεί η αντιπολίτευση διαδήλωση έξω από την οικογενειακή εστία υπουργού με τις πολιτικές αποφάσεις του οποίου διαφωνεί μερίδα πολιτών;</p>,
 <p>Η <strong>διαδήλωση αντιεμβολιαστών</strong> έξω από την κατοικία του υπουργού Υγείας Θ. Πλεύρη, το απόγευμα της Δευτέρας 27 Ιουνίου, υποδηλώνει την ακόμη μεγαλύτερη κατρακύλα της πολιτικής αντιπαράθεσης, με τα κόμματα της αντιπολίτευσης να μην αντιδρούν ακαριαία στ

In [33]:
p_texts_list = []                      # δημιουργία κενής λίστας για να αποθηκεύσουμε το text κάθε παραγράφου
paragraphs = article.find_all('p')     # λίστα με όλες τις παραγράφους
for p in paragraphs:                   # για κάθε μια από τις παραγράφους στη λίστα
  p_texts_list.append(p.text)          # πάρε το .text και πρόσθεσέ το στη λίστα με τα texts των παραγράφων
full_text = ' '.join(p_texts_list)     # συνένωση των texts των παραγράφων
full_text = "".join(full_text.splitlines())
full_text

'Η διαδήλωση αντιεμβολιαστών έξω από την κατοικία του υπουργού Υγείας Θ. Πλεύρη, το απόγευμα της Δευτέρας 27 Ιουνίου, υποδηλώνει την ακόμη μεγαλύτερη κατρακύλα της πολιτικής αντιπαράθεσης, με τα κόμματα της αντιπολίτευσης να μην αντιδρούν ακαριαία στην καταδίκη της ενέργειας. Θα ζήσουμε προεκλογική περίοδο με διαμαρτυρόμενους έξω από τα σπίτια των υπουργών και την σύζυγο με τα παιδιά τους εντός κατοικίας; Είναι μέρος της πολιτικής αντιπαράθεσης η στοχοποίηση της οικογένειας του υπουργού; Επικροτεί η αντιπολίτευση διαδήλωση έξω από την οικογενειακή εστία υπουργού με τις πολιτικές αποφάσεις του οποίου διαφωνεί μερίδα πολιτών; Η διαδήλωση αντιεμβολιαστών έξω από την κατοικία του υπουργού Υγείας Θ. Πλεύρη, το απόγευμα της Δευτέρας 27 Ιουνίου, υποδηλώνει την ακόμη μεγαλύτερη κατρακύλα της πολιτικής αντιπαράθεσης, με τα κόμματα της αντιπολίτευσης να μην αντιδρούν ακαριαία στην καταδίκη της ενέργειας. Την κατοικία του υπουργού Υγείας πολιόρκησαν αντιεμβολιαστές, ενδεχομένως και οι του υγειονο

In [34]:
full_articles_list = []

for article_url in skai_teasers_df['url']:
  response = requests.get(article_url)
  doc = BeautifulSoup(response.text, 'html.parser')

  #*** ΔΩΣΤΕ ΤΗ ΔΙΚΗ ΣΑΣ ΤΙΜΗ στo article που βρήκατε παραπάνω***
  article = doc.find('body', {'class': 'skinEnabled'})

  full_article_dict = {}

  # website
  full_article_dict['site'] = "skai.gr"

  # url
  full_article_dict['url'] = article_url

  #section
  try:
    section = doc.find('div', {'class': 'bold uppercase'}).text
    full_article_dict['section'] = section
  except:
    try:
      section = doc.find('div', {'class': 'left bold uppercase'}).text
      full_article_dict['section'] = section
    except:
      full_article_dict['section'] = ''
      pass

  #title
  try:
    title = article.find('article').find('h1').text
    full_article_dict['title'] = title
  except:
    full_article_dict['title'] = ''
    pass

  # date
  try:
    date = article.find('time')['datetime']
    full_article_dict['date'] = date
  except:
    full_article_dict['date'] = ''
    pass

  # author
  try:
    author = article.find('a', {'class': 'author bold'}).text
    full_article_dict['author'] = author
  except:
    full_article_dict['author'] = ''
    pass

  # full_text
  try:
    p_texts_list = []
    paragraphs = article.find_all('p')
    for p in paragraphs:
      p_texts_list.append(p.text)
    full_text = ' '.join(p_texts_list)
    full_text = "".join(full_text.splitlines())
    full_article_dict['full_text'] = full_text
  except:
    full_article_dict['full_text'] = ''
    pass

  full_articles_list.append(full_article_dict)
  time.sleep(1)

In [35]:
skai_full_articles_df = pd.DataFrame(full_articles_list)
skai_full_articles_df

,site,url,section,title,date,author,full_text
0,skai.gr,https://www.skai.gr/news/greece/volos-ereyna-g...,Αρχική / Ελλάδα,Βόλος: Έρευνα για τον αντιεμβολιαστή γιατρό πο...,2022-12-08T08:09:51+02:00,,Εξέταζε διαδικτυακά τους ασθενείς και τους απέ...
1,skai.gr,https://www.skai.gr/news/world/nea-zilandia-go...,Αρχική / Κόσμος,Νέα Ζηλανδία: Γονείς αρνούνται να δοθεί αίμα ε...,2022-12-06T08:54:02+02:00,,"Το τεσσάρων μηνών βρέφος πάσχει, σύμφωνα με τη..."
2,skai.gr,https://www.skai.gr/news/greece/amvrosios-kata...,Αρχική / Ελλάδα,Αμβρόσιος κατά Πλεύρη για ανεμβολίαστους υγειο...,2022-10-26T11:09:40+03:00,,Ο κ. Αμβρόσιος τα βάζει με την κυβέρνηση για τ...
3,skai.gr,https://www.skai.gr/news/health/dw-treis-aione...,Αρχική / Υγεία,DW: Τρεις αιώνες εμβόλια - Από την ευλογιά στο...,2022-08-24T12:40:51+03:00,,"Με ιστορία τριών αιώνων, ο εμβολιασμός έχει εξ..."
4,skai.gr,https://www.skai.gr/news/world/aystria-thrinos...,Αρχική / Κόσμος,Αυστρία: Θρήνος για γιατρό που αυτοκτόνησε λόγ...,2022-07-30T13:34:11+03:00,,Η Λίζα-Μαρία Κέλερμερ έδινε συχνά συνεντεύξεις...
5,skai.gr,https://www.skai.gr/news/greece/se-syllipseis-...,Αρχική / Ελλάδα,Σε συλλήψεις μετατράπηκαν οι 38 προσαγωγές αντ...,2022-07-08T04:48:37+03:00,,Σύμφωνα με πηγές της αστυνομίας και οι 38 μετα...
6,skai.gr,https://www.skai.gr/news/greece/prosagoges-ant...,Αρχική / Ελλάδα,Προσαγωγές αντιεμβολιαστών έξω από το σπίτι το...,2022-07-07T20:46:59+03:00,,Ανάμεσά τους και υγειονομικοί Του Μάκη Συνοδιν...
7,skai.gr,https://www.skai.gr/news/health/koronoios-ipa-...,Αρχική / Υγεία,Κορωνοϊός: Πώς η παραπληροφόρηση έδωσε ώθηση σ...,2022-07-04T10:53:48+03:00,,Τα αντιεμβολιαστικά μηνύματα στα μέσα κοινωνικ...
8,skai.gr,https://www.skai.gr/news/politics/pleyris-diad...,Αρχική / Πολιτική,«Αντιεμβολιαστές» ξανά έξω από το σπίτι του Θά...,2022-06-30T22:45:59+03:00,,Σε ανάρτησή του στο Τwitter αναφέρει: «Δυστυχώ...
9,skai.gr,https://www.skai.gr/news/greece/koronoios-pley...,Αρχική / Ελλάδα,Πλεύρης σε ΣΚΑΪ: Δεν θα λάβουμε μέτρα- Σε λογι...,2022-06-29T08:29:01+03:00,,Ο υπουργός Υγείας τόνισε ότι αυτή τη στιγμή τα...


In [36]:
skai_full_articles_df.to_csv('/content/temp/skai_full_articles.csv', index=False)